In [50]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score

import numpy as np
import tensorflow as tf

import keras
import os
import sys

sys.path.append('..')
import data
import model
import preprocessing
import utils

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
params = utils.yaml_to_dict(os.path.join('..','config.yml'))
params['data_dir'] = os.path.join('..',params['data_dir'])
params['model_dir'] = os.path.join('..','experiments-weights','Experimento1-Configuracion12')
params['batch_size'] = 8
params['num_epochs'] = 30
params['learning_rate'] = 1

## Primer paso: preprocesar los datos 
Hacerlo solo una vez

In [52]:
#preprocessing.download_data(params)
#preprocessing.extract_data(params)
#preprocessing.generate_spectogram_images(params)
#preprocessing.make_id_label_map(params)
preprocessing.split_data(params)

## Segund Paso: instanciar el modelo

In [53]:
inputs = tf.keras.layers.Input(shape=(110,480, 3))
net = model.ModelArchitecture(num_classes=params['num_classes'])
x = net(inputs, training=False)
net.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 55, 240, 3)        444       
_________________________________________________________________
batch_normalization_8 (Batch (None, 55, 240, 3)        12        
_________________________________________________________________
activation_12 (Activation)   (None, 55, 240, 3)        0         
_________________________________________________________________
global_average_pooling2d_4 ( (None, 3)                 0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               512       
_________________________________________________________________
batch_normalization_9 (Batch (None, 128)               512       
_________________________________________________________________
activation_13 (Activation)   (None, 128)               0         
__________

## Tercer paso: crear el generador

In [54]:
print('Data in train')
train_generator = data.make_datagenerator(params, mode='training')
print('Data in validation')
val_generator = data.make_datagenerator(params,mode='validation')

Data in train
Found 2960 images.
Data in validation
Found 30 images.


## Cuarto paso: correr el modelo

In [55]:
if not os.path.exists(params['model_dir']):
    os.makedirs(params['model_dir'])

In [56]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    os.path.join(params['model_dir'], 'tf_ckpt'), 
    save_weights_only=True, 
    verbose=1,
    period=5)

tb_callback = tf.keras.callbacks.TensorBoard(
    os.path.join(params['model_dir'], 'logs'))

optimizer = tf.keras.optimizers.Adam(params['learning_rate'])

steps_per_epoch = train_generator.n // params['batch_size']
validation_steps = val_generator.n // params['batch_size']

net.compile(optimizer=optimizer, loss=params['loss'], metrics=['sparse_categorical_accuracy'])
net.fit_generator(
    train_generator, 
    steps_per_epoch=steps_per_epoch, 
    epochs=params['num_epochs'],
    workers=4,
    validation_data=val_generator, 
    validation_steps=validation_steps,
    callbacks=[cp_callback,tb_callback])

Epoch 1/30
370/370 [==============================] - 23s 63ms/step - loss: 14.3109 - sparse_categorical_accuracy: 0.1095 - val_loss: 13.4317 - val_sparse_categorical_accuracy: 0.1667
Epoch 2/30
370/370 [==============================] - 22s 59ms/step - loss: 14.3375 - sparse_categorical_accuracy: 0.1105 - val_loss: 13.4317 - val_sparse_categorical_accuracy: 0.1667
Epoch 3/30
370/370 [==============================] - 22s 60ms/step - loss: 14.3375 - sparse_categorical_accuracy: 0.1105 - val_loss: 13.4317 - val_sparse_categorical_accuracy: 0.1667
Epoch 4/30
370/370 [==============================] - 21s 58ms/step - loss: 14.3375 - sparse_categorical_accuracy: 0.1105 - val_loss: 13.4317 - val_sparse_categorical_accuracy: 0.1667
Epoch 5/30
369/370 [============================>.] - ETA: 0s - loss: 14.3327 - sparse_categorical_accuracy: 0.1108
Epoch 00005: saving model to ..\experiments-weights\Experimento1-Configuracion12\tf_ckpt

Consider using a TensorFlow optimizer from `tf.train`.
370

370/370 [==============================] - 28s 75ms/step - loss: 14.3375 - sparse_categorical_accuracy: 0.1105 - val_loss: 13.4317 - val_sparse_categorical_accuracy: 0.1667
Epoch 21/30
370/370 [==============================] - 26s 70ms/step - loss: 14.3375 - sparse_categorical_accuracy: 0.1105 - val_loss: 13.4317 - val_sparse_categorical_accuracy: 0.1667
Epoch 22/30
370/370 [==============================] - 26s 72ms/step - loss: 14.3375 - sparse_categorical_accuracy: 0.1105 - val_loss: 13.4317 - val_sparse_categorical_accuracy: 0.1667
Epoch 23/30
370/370 [==============================] - 26s 71ms/step - loss: 14.3375 - sparse_categorical_accuracy: 0.1105 - val_loss: 13.4317 - val_sparse_categorical_accuracy: 0.1667
Epoch 24/30
370/370 [==============================] - 27s 74ms/step - loss: 14.3375 - sparse_categorical_accuracy: 0.1105 - val_loss: 13.4317 - val_sparse_categorical_accuracy: 0.1667
Epoch 25/30
369/370 [============================>.] - ETA: 0s - loss: 14.3381 - sparse

In [57]:
#fpr, tpr, thresholds = roc_curve(y, probs)

In [58]:
# Quintero paso: evaluar el modelo

In [59]:
params['batch_size'] = val_generator.n
params['shuffle'] = False
val_generator = data.make_datagenerator(params,mode='validation')

Found 30 images.


In [60]:
Y_pred = net.predict_generator(val_generator, steps=1)
y_pred = np.argmax(Y_pred, axis=1)

In [61]:
print('Confusion Matrix')
print(confusion_matrix(val_generator.data, y_pred))
target_names = ["arranca", "acelera", "pita", "izquierda","frena", "detente", "retrocede", "gira", "avanza", "derecha"]
print(classification_report(val_generator.data, y_pred, target_names=target_names))

Confusion Matrix
[[0 2 0 0 0 0 0 0 0 0]
 [0 4 0 0 0 0 0 0 0 0]
 [0 4 0 0 0 0 0 0 0 0]
 [0 4 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [0 2 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0]
 [0 2 0 0 0 0 0 0 0 0]]
              precision    recall  f1-score   support

     arranca       0.00      0.00      0.00         2
     acelera       0.13      1.00      0.24         4
        pita       0.00      0.00      0.00         4
   izquierda       0.00      0.00      0.00         4
       frena       0.00      0.00      0.00         3
     detente       0.00      0.00      0.00         3
   retrocede       0.00      0.00      0.00         3
        gira       0.00      0.00      0.00         2
      avanza       0.00      0.00      0.00         3
     derecha       0.00      0.00      0.00         2

   micro avg       0.13      0.13      0.13        30
   macro avg       0.01      0.10      0.02        30
weighted avg       0.02      0.13      0.03     